---
title: "25-因果推断中的线性回归"
author: "Simonzhou"
date: "2025-07-29"
date-modified: today
format: 
    html:
        code-fold: true
        code-line-numbers: true
        code-highlight: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 3
---

In [1]:
import stata_setup
stata_setup.config('C:/Program Files/Stata18', 'mp', splash=False)

# 线性回归

这里将使用一个案例演示 regress 和 teffects ra 的用法。

案例数据来自 “中国家庭追踪数据”（China Family Panel Studies, CFPS）2010年的数据，该数据集由中国社会科学院和北京大学共同开发，旨在收集中国家庭的社会、经济和人口信息。

数据总共包括 4137 名年龄在25-55 岁之间的成年人，集体如下：

In [2]:
%%stata
use "../../Data/causal-inference/cfps2010.dta", clear
* 查看数据随机十个数据
list in 1/10


. use "../../Data/causal-inference/cfps2010.dta", clear

. * 查看数据随机十个数据
. list in 1/10

     +------------------------------------------------------------------------+
  1. |       pid | provcd | gender | age | age2 |   age3 |    lninc | college |
     | 110003101 | 北京市 |     男 |  39 | 1521 |  59319 | 11.51293 |      否 |
     |------------------------------------------------------------------------|
     |        hukou      |     sibling      |     race      |      fmedu      |
     |     农村户口      |          否      |       是      |       缺失      |
     +------------------------------------------------------------------------+

     +------------------------------------------------------------------------+
  2. |       pid | provcd | gender | age | age2 |   age3 |    lninc | college |
     | 110005102 | 北京市 |     男 |  48 | 2304 | 110592 | 9.392662 |      否 |
     |------------------------------------------------------------------------|
     |        hukou      |     sibling      |   

## 简要的查看一下数据

In [3]:
%%stata
describe


Contains data from ../../Data/causal-inference/cfps2010.dta
 Observations:         4,137                  
    Variables:            12                  5 Aug 2022 19:25
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
pid             double  %12.0g     pid        个人id
provcd          double  %24.0g     provcd     省国标码
gender          double  %12.0g     gender     性别
age             float   %9.0g                 年龄
age2            float   %9.0g                 年龄平方
age3            float   %9.0g                 年龄三次方
lninc           float   %9.0g                 收入对数
college         double  %9.0g      yesorno    是否上大学
hukou           double  %12.0g     hukou      3岁时户口性质
sibling         float   %9.0g      yesorno    是否独生子女
race            double  %9.0g      yesorno    是否汉族
fme

分析变量为收入对数 `lninc`，核心自变量为是否上大学 `college`，该变量取值为 1 表示研究对象读过大学，取值为 0 表示没有读过大学。

基于上述两个变量，可以进行一元线性回归，具体如下：

In [4]:
%%stata
regress lninc college


      Source |       SS           df       MS      Number of obs   =     4,137
-------------+----------------------------------   F(1, 4135)      =    508.22
       Model |  671.883317         1  671.883317   Prob > F        =    0.0000
    Residual |  5466.56371     4,135  1.32202266   R-squared       =    0.1095
-------------+----------------------------------   Adj R-squared   =    0.1092
       Total |  6138.44703     4,136  1.48415063   Root MSE        =    1.1498

------------------------------------------------------------------------------
       lninc | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .823612   .0365338    22.54   0.000     .7519861     .895238
       _cons |   9.353189   .0230235   406.25   0.000     9.308051    9.398327
------------------------------------------------------------------------------


从分析结果可以看出，在不控制任何变量的情况下，college 的回归系数为 0.824 ，其标准误为 0.037。

用回归系数除以标准误即可得到 t 统计量的值，为 22.54，它的 P 值在小数点后三位都是 0，说明 college 对 lninc 的影响是非常显著的。

除此之外，Stata 还报告了置信度为 95% 的置信区间为 \[0.752, 0.896\]。

需要注意的是，上述标准误以及统计检验结果都是基于独立同分布假定计算的，因此在实际应用中需要谨慎解读。

在显示条件下，该假定通常不能得到满足，因此，建议使用更加稳健的标准误和统计检验。

## 使用稳健标准误

使用怀特异方差稳健标准误来调整异方差的影响，使用方法在 `regress` 命令后使用选项 `vce(robust)`。

In [5]:
%%stata
regress lninc college, vce(robust)


Linear regression                               Number of obs     =      4,137
                                                F(1, 4135)        =     582.05
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1095
                                                Root MSE          =     1.1498

------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .823612   .0341385    24.13   0.000     .7566823    .8905418
       _cons |   9.353189   .0256882   364.10   0.000     9.302826    9.403552
------------------------------------------------------------------------------


从结果中可以看到，使用稳健标准误后，college 的回归系数仍然为 0.824，但其标准误变为 0.034，相比之前略有下降，t 统计量变为 24.13，P 值在小数点后三位都是 0，说明 college 对 lninc 的影响仍然是非常显著的。

由于异方差条件下的标准误和统计检验结果更加可信，因此建议用户在使用 `regress` 命令时，使用选项 `vce(robust)`。

## 多阶段整群抽样

用户还可以使用选项 vce(cluster clustvar) 进一步放松误差项之间相互独立的假定。

CFPS 采用的是以地理区域为抽样单位的多阶段整群抽样，这样会导致同一个抽样单位中的样本存在自相关。

为了降低自相关的影响，以调查样本所在省（provcd）为聚类变量，使用选项 `vce(cluster provcd)`，计算聚类稳健标准误，命令和结果如下：

In [3]:
%%stata
regress lninc college, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(1, 24)          =     271.17
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1095
                                                Root MSE          =     1.1498

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .823612   .0500155    16.47   0.000     .7203851     .926839
       _cons |   9.353189   .1084703    86.23   0.000     9.129317    9.577061
------------------------------------------------------------------------------


在使用 `vce(cluster provcd)` 后，college 的回归系数仍然为 0.824，但其标准误变为 0.050，相比之前略有上升，t 统计量变为 16.47，P 值在小数点后三位都是 0，说明 college 对 lninc 的影响仍然是非常显著的。

在这个聚类稳健标准误的结果中，统计检验结果没有受到影响，相比之前，使用聚类稳健标准误的结果最为稳健，因此，后续都将使用此类标准误的分析结果。

上述分析都只包含 college 这一个自变量，在这种情况下，进行因果推断必须满足严格的可忽略性假定。即要求大学学历在研究对象中随机分配，这个假定显然与中国的实际情况相差很远，因此，上述分析结果仅能代表上大学的人和不上大学的人在收入对数上的平均差异，不能解释为大学对收入的因果影响。

## 控制变量

为了使回归分析结果具有因果解释的效应，需要在模型中纳入控制变量，以尽可能地满足条件独立假定。

这里我们认为，就这个研究问题来说，研究对象出生时的户口性质（hukou）是一个非常重要的混杂变量。

一方面，由于教育资源分布的城乡不均，城市人比农村人拥有更多考上大学的机会；另一方面，因为劳动力市场上的城乡分割，城市人也比农村人拥有更好的收入前景。

综合这两个方面，有必要在模型分析时控制 hukou 变量。

In [4]:
%%stata
regress lninc college hukou, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(2, 24)          =     219.69
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1169
                                                Root MSE          =     1.1451

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .798281   .0451727    17.67   0.000     .7050491     .891513
       hukou |   .2155333   .0611855     3.52   0.002     .0892526    .3418141
       _cons |    9.27541   .0993474    93.36   0.000     9.070367    9.480453
--------------